In [ ]:
import pandas as pd
import numpy as np

import torch

import os

In [ ]:
# Set the fold and the device used to make the prediction

current_best_model = "../models/fold0_fella_0.8388671875.pth"
current_fold = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(0)

In [ ]:
# Load the raw data

X_train_raw = pd.read_csv('../X_train.csv', index_col='id')
X_test_raw = pd.read_csv('../X_test.csv', index_col='id')
y_train_raw = pd.read_csv('../y_train.csv', index_col=0)['y'].to_numpy()

In [ ]:
# Expand all signals to 18000 length

from myutils import multi_features

os.environ['OMP_NUM_THREADS'] = "1"

train_expanded = multi_features(X_train_raw, n_cores=128)
test_expanded = multi_features(X_test_raw, n_cores=128)

In [ ]:
X_train = np.expand_dims(train_expanded, 1)
X_test = np.expand_dims(test_expanded, 1)
y_train = y_train_raw

In [ ]:
from deepnet import MyDataset
from torch.utils.data import DataLoader

batch_size=32

train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, np.zeros(X_test.shape[0]))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
from deepnet import DeepNet

model = DeepNet(in_channels=18000, n_classes=4).to(device)
model.load_state_dict(torch.load(current_best_model))
model.dense = torch.nn.Identity()
model.to(device)
model.eval()

In [ ]:
train_features = []
for batch_idx, batch in enumerate(train_dataloader):
    input_x, input_y = tuple(t.to(device) for t in batch)
    pred = model(input_x)
    n_pred = pred.cpu().data.numpy()
    train_features += [x for x in n_pred]
    
print(len(train_features))

In [ ]:
test_features = []
for batch_idx, batch in enumerate(test_dataloader):
    input_x, input_y = tuple(t.to(device) for t in batch)
    pred = model(input_x)
    n_pred = pred.cpu().data.numpy()
    test_features += [x for x in n_pred]
    
print(len(test_features))

In [ ]:
fella_ml_training = np.array(train_features)
fella_ml_test = np.array(test_features)

In [ ]:
np.savetxt(f"fella_ml_training_features{current_fold}.txt", fella_ml_training, delimiter=',')
np.savetxt(f"fella_ml_test_features{current_fold}.txt", fella_ml_test, delimiter=',')